Classification Model

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

In [2]:
df_pluviometricos = pd.read_csv("../dados/dados_pluviometricos_geo_19_01_2018.csv")

In [3]:
df_pluviometricos.head()

,hex,NOME,geometry,lat,long,latitude,longitude,date,valorMedido,value_last_3_days,value_last_7_days
0,88a804cb05fffff,SANTA FELICIDADE,POLYGON ((-49.32705987407714 -25.3834767821367...,-49.327917,-25.378939,-49.327917,-25.378939,2018-01-19,51.479421,80.070905,117.150287
1,88a804cb07fffff,SANTA FELICIDADE,POLYGON ((-49.33292565560358 -25.3771150332313...,-49.333783,-25.372577,-49.333783,-25.372577,2018-01-19,40.814741,45.489126,75.415248
2,88a804cb09fffff,SANTA FELICIDADE,POLYGON ((-49.33951229851928 -25.3916145095337...,-49.340370,-25.387078,-49.340370,-25.387078,2018-01-19,30.343360,21.292583,37.365156
3,88a804cb0dfffff,SANTA FELICIDADE,POLYGON ((-49.33035322747818 -25.3907265222753...,-49.331211,-25.386190,-49.331211,-25.386190,2018-01-19,45.722535,63.906104,93.874085
4,88a804cb29fffff,SANTA FELICIDADE,POLYGON ((-49.317900452100716 -25.382587743287...,-49.318758,-25.378050,-49.318758,-25.378050,2018-01-19,53.341340,80.677513,119.162033


In [11]:
df_curva_niveis = pd.read_csv("../dados/dados_curva_niveis.csv")

In [12]:
df_curva_niveis

,date,score,hex,NOME,geometry,index_right,Id,Layer,Elevation
0,2020-05-18,100,88a831acc7fffff,BAIRRO NOVO,POLYGON ((-49.27455781428023 -25.5376384480459...,26422,0,RE_CURVA_MESTRA,905.0
1,2020-03-11,100,88a831ace1fffff,BAIRRO NOVO,POLYGON ((-49.256230009114155 -25.535862749251...,26422,0,RE_CURVA_MESTRA,905.0
2,2020-07-12,100,88a831acc5fffff,BAIRRO NOVO,POLYGON ((-49.268688398995586 -25.543987102761...,26422,0,RE_CURVA_MESTRA,905.0
3,2020-04-25,99,88a831acedfffff,BAIRRO NOVO,POLYGON ((-49.25035995958426 -25.5422110506431...,26422,0,RE_CURVA_MESTRA,905.0
4,2020-08-23,100,88a831acedfffff,BAIRRO NOVO,POLYGON ((-49.25035995958426 -25.5422110506431...,26422,0,RE_CURVA_MESTRA,905.0
...,...,...,...,...,...,...,...,...,...
62282,2020 -04 -23,20,88a831a509fffff,TATUQUARA,POLYGON ((-49.32654950366851 -25.6325161091234...,14906,0,RE_CURVA_INTERMEDIARIA,867.0
62283,2020 -04 -23,20,88a831a509fffff,TATUQUARA,POLYGON ((-49.32654950366851 -25.6325161091234...,31060,0,RE_CURVA_INTERMEDIARIA,868.0
62284,2020 -04 -23,20,88a831a509fffff,TATUQUARA,POLYGON ((-49.32654950366851 -25.6325161091234...,14928,0,RE_CURVA_INTERMEDIARIA,867.0
62285,2020 -04 -23,20,88a831a509fffff,TATUQUARA,POLYGON ((-49.32654950366851 -25.6325161091234...,15398,0,RE_CURVA_INTERMEDIARIA,867.0


In [15]:
df_curva_niveis[df_curva_niveis["hex"] == "88a831acc7fffff"]

,hex,geometry,Elevation,score
510,88a831acc7fffff,POLYGON ((-49.27455781428023 -25.5376384480459...,897.076923,100.0


In [14]:
df_curva_niveis = df_curva_niveis.groupby(["hex", "geometry"], as_index=False)["Elevation", "score"].mean()

/var/folders/ld/8dhf4gy93xbd1ybmf6nljm38dn1p1r/T/ipykernel_46111/975208766.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_curva_niveis = df_curva_niveis.groupby(["hex", "geometry"], as_index=False)["Elevation", "score"].mean()


Preparar o dataset de alagamentos (o target)

In [16]:
df_alagamentos = pd.read_csv("../dados/alagamentos_19_01_2018.csv")

In [17]:
df_alagamentos.head()

,Unnamed: 0,index_left,N_,ENDERECO,BAIRRO,CIDADE,NUCLEO,PROTOCOLO,SECRETARIA,DATA,...,HORA,FLAGRANTE,MODIFICADO,ORIGINAL,NATUREZA,date,score,hex,NOME,geometry
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Nenhum,2020-05-18,100,88a831acc7fffff,BAIRRO NOVO,POLYGON ((-49.27455781428023 -25.5376384480459...
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Nenhum,2020-03-11,100,88a831ace1fffff,BAIRRO NOVO,POLYGON ((-49.256230009114155 -25.535862749251...
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Nenhum,2020-04-01,100,88a831acebfffff,BAIRRO NOVO,POLYGON ((-49.26539407770731 -25.5367509063624...
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Nenhum,2020-07-12,100,88a831acc5fffff,BAIRRO NOVO,POLYGON ((-49.268688398995586 -25.543987102761...
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Nenhum,2020-04-25,99,88a831acedfffff,BAIRRO NOVO,POLYGON ((-49.25035995958426 -25.5422110506431...


In [18]:
df_alagamentos[df_alagamentos["geometry"] == "POLYGON ((-49.27455781428023 -25.537638448045946, -49.270405175714615 -25.534930506545987, -49.27126354011573 -25.53040181434035, -49.27627444225334 -25.52858115114508, -49.2804269592526 -25.53128917959227, -49.27956869568991 -25.535817784277132, -49.27455781428023 -25.537638448045946))"]

,Unnamed: 0,index_left,N_,ENDERECO,BAIRRO,CIDADE,NUCLEO,PROTOCOLO,SECRETARIA,DATA,...,HORA,FLAGRANTE,MODIFICADO,ORIGINAL,NATUREZA,date,score,hex,NOME,geometry
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Nenhum,2020-05-18,100,88a831acc7fffff,BAIRRO NOVO,POLYGON ((-49.27455781428023 -25.5376384480459...


In [19]:
df_alagamentos["flooded"] = df_alagamentos["NATUREZA"]
df_alagamentos

,Unnamed: 0,index_left,N_,ENDERECO,BAIRRO,CIDADE,NUCLEO,PROTOCOLO,SECRETARIA,DATA,...,FLAGRANTE,MODIFICADO,ORIGINAL,NATUREZA,date,score,hex,NOME,geometry,flooded
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020-05-18,100,88a831acc7fffff,BAIRRO NOVO,POLYGON ((-49.27455781428023 -25.5376384480459...,Nenhum
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020-03-11,100,88a831ace1fffff,BAIRRO NOVO,POLYGON ((-49.256230009114155 -25.535862749251...,Nenhum
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020-04-01,100,88a831acebfffff,BAIRRO NOVO,POLYGON ((-49.26539407770731 -25.5367509063624...,Nenhum
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020-07-12,100,88a831acc5fffff,BAIRRO NOVO,POLYGON ((-49.268688398995586 -25.543987102761...,Nenhum
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020-04-25,99,88a831acedfffff,BAIRRO NOVO,POLYGON ((-49.25035995958426 -25.5422110506431...,Nenhum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020 -07 -31,59,88a831a551fffff,TATUQUARA,POLYGON ((-49.34487490397486 -25.6342766856842...,Nenhum
880,880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020 -04 -23,47,88a831a55bfffff,TATUQUARA,POLYGON ((-49.354037100313235 -25.635156048888...,Nenhum
881,881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020 -03 -24,55,88a831a555fffff,TATUQUARA,POLYGON ((-49.335712371609894 -25.633396705770...,Nenhum
882,882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Nenhum,2020 -03 -21,23,88a831a465fffff,TATUQUARA,POLYGON ((-49.357331758706884 -25.642378641771...,Nenhum


In [20]:
df_alagamentos = df_alagamentos[["flooded", "date", "hex", "NOME", "geometry"]]

In [21]:
df_alagamentos["flooded"].unique()

array(['Nenhum', 'Alagamento'], dtype=object)

Preparar a elevação

In [28]:
# df = df_curva_niveis.join(df_alagamentos)
df_merged = pd.merge(df_alagamentos, df_curva_niveis, on="geometry")

In [29]:
df_merged["elevation"] = df_merged["Elevation"]
df_merged["hex"] = df_merged["hex_x"]
df_merged["score_elevation"] = df_merged["score"]
df_merged = df_merged[["flooded", "date", "hex", "geometry", "score_elevation", "elevation"]]

In [30]:
df_merged[df_merged["flooded"] == "Alagamento"]

,flooded,date,hex,geometry,score_elevation,elevation
210,Alagamento,2020 -08 -28,88a8313609fffff,POLYGON ((-49.25736804693868 -25.4806112919251...,100.0,887.548387
318,Alagamento,2020 -08 -25,88a831375dfffff,POLYGON ((-49.205676071931734 -25.482504410546...,99.5,900.590909
319,Alagamento,2020 -08 -28,88a831375dfffff,POLYGON ((-49.205676071931734 -25.482504410546...,99.5,900.590909
334,Alagamento,2020 -04 -21,88a8313741fffff,POLYGON ((-49.19063894381699 -25.4879613789410...,99.5,874.330769
335,Alagamento,2020 -08 -09,88a8313741fffff,POLYGON ((-49.19063894381699 -25.4879613789410...,99.5,874.330769
382,Alagamento,2020 -04 -05,88a8313719fffff,POLYGON ((-49.21412388710045 -25.4625550791301...,100.0,896.900000
433,Alagamento,2020 -03 -21,88a804db37fffff,POLYGON ((-49.337252107133 -25.502189410110187...,100.0,888.902439
447,Alagamento,2020 -08 -29,88a804db3dfffff,POLYGON ((-49.33467929396914 -25.5157788937393...,100.0,888.863636
665,Alagamento,2020 -05 -13,88a804d921fffff,POLYGON ((-49.296592857777654 -25.470571367177...,100.0,922.901961
793,Alagamento,2020 -06 -06,88a804d9adfffff,POLYGON ((-49.331795060569874 -25.432434059539...,98.0,948.830645


In [38]:
df_merged.drop_duplicates()

,flooded,date,hex,geometry,score_elevation,elevation
0,Nenhum,2020-05-18,88a831acc7fffff,POLYGON ((-49.27455781428023 -25.5376384480459...,100.0,897.076923
1,Nenhum,2020-03-11,88a831ace1fffff,POLYGON ((-49.256230009114155 -25.535862749251...,100.0,894.491803
2,Nenhum,2020-04-01,88a831acebfffff,POLYGON ((-49.26539407770731 -25.5367509063624...,100.0,888.318182
3,Nenhum,2020-07-12,88a831acc5fffff,POLYGON ((-49.268688398995586 -25.543987102761...,100.0,895.594595
4,Nenhum,2020-04-25,88a831acedfffff,POLYGON ((-49.25035995958426 -25.5422110506431...,99.5,887.250000
...,...,...,...,...,...,...
879,Nenhum,2020 -07 -31,88a831a551fffff,POLYGON ((-49.34487490397486 -25.6342766856842...,59.0,882.213333
880,Nenhum,2020 -04 -23,88a831a55bfffff,POLYGON ((-49.354037100313235 -25.635156048888...,47.0,885.430108
881,Nenhum,2020 -03 -24,88a831a555fffff,POLYGON ((-49.335712371609894 -25.633396705770...,55.0,869.081481
882,Nenhum,2020 -03 -21,88a831a465fffff,POLYGON ((-49.357331758706884 -25.642378641771...,23.0,875.209877


In [41]:
df_merged[df_merged["hex"] == "88a831375dfffff"]

,flooded,date,hex,geometry,score_elevation,elevation
318,Alagamento,2020 -08 -25,88a831375dfffff,POLYGON ((-49.205676071931734 -25.482504410546...,99.5,900.590909
319,Alagamento,2020 -08 -28,88a831375dfffff,POLYGON ((-49.205676071931734 -25.482504410546...,99.5,900.590909


In [40]:
first == seccond

True

In [42]:
df_merged = pd.merge(df_merged, df_pluviometricos, on="geometry")

In [43]:
df_merged.head()

,flooded,date_x,hex_x,geometry,score_elevation,elevation,hex_y,NOME,lat,long,latitude,longitude,date_y,valorMedido,value_last_3_days,value_last_7_days
0,Nenhum,2020-07-12,88a831acc5fffff,POLYGON ((-49.268688398995586 -25.543987102761...,100.0,895.594595,88a831acc5fffff,BAIRRO NOVO,-49.269547,-25.539459,-49.269547,-25.539459,2018-01-19,37.478475,48.776552,75.025687
1,Nenhum,2020 -06 -04,88a831acc1fffff,POLYGON ((-49.27785212064187 -25.5448742055090...,99.5,914.868421,88a831acc1fffff,BAIRRO NOVO,-49.278710,-25.540346,-49.278710,-25.540346,2018-01-19,28.460733,37.512889,60.644139
2,Nenhum,2020 -03 -09,88a831acc1fffff,POLYGON ((-49.27785212064187 -25.5448742055090...,99.5,914.868421,88a831acc1fffff,BAIRRO NOVO,-49.278710,-25.540346,-49.278710,-25.540346,2018-01-19,28.460733,37.512889,60.644139
3,Nenhum,2020 -08 -14,88a831accdfffff,POLYGON ((-49.27198275252102 -25.5512224228430...,99.5,908.909091,88a831accdfffff,BAIRRO NOVO,-49.272841,-25.546695,-49.272841,-25.546695,2018-01-19,30.337897,45.561371,69.335039
4,Nenhum,2020 -05 -09,88a831accdfffff,POLYGON ((-49.27198275252102 -25.5512224228430...,99.5,908.909091,88a831accdfffff,BAIRRO NOVO,-49.272841,-25.546695,-49.272841,-25.546695,2018-01-19,30.337897,45.561371,69.335039


In [45]:

df_merged["value_precipitation"] = df_merged["valorMedido"]
df_merged["value_precipitation_3_days"] = df_merged["value_last_3_days"]
df_merged["value_precipitation_7_days"] = df_merged["value_last_7_days"]
df_merged["hex"] = df_merged["hex_x"]
df_merged["date"] = df_merged["date_x"]
df_merged = df_merged[["flooded", "date", "hex", "geometry", "score_elevation", "elevation", "value_precipitation", "value_precipitation_3_days", "value_precipitation_7_days"]]

Rodar modelo de classificação

In [ ]:
import pandas as pd
import geopandas as gpd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

features = ['elevation', 'score_elevation', 'value_precipitation', 'value_precipitation_3_days', 'value_precipitation_7_days']
target = 'flooded'

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_merged[features], df_merged[target], test_size=0.2, random_state=42)

# Criar e treinar a árvore de decisão
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Avaliar o desempenho do modelo no conjunto de teste
score = clf.score(X_test, y_test)
print("Acurácia do modelo:", score)


In [46]:

features = df_merged[['elevation', 'score_elevation', 'value_precipitation', 'value_precipitation_3_days', 'value_precipitation_7_days']]
target = df_merged[['flooded']]

In [56]:
train_features, test_features, train_labels, test_labels = train_test_split(features, target, test_size=0.5)

In [57]:
rf = RandomForestClassifier()
rf.fit(train_features, train_labels)

/var/folders/ld/8dhf4gy93xbd1ybmf6nljm38dn1p1r/T/ipykernel_46111/2852299628.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_features, train_labels)


RandomForestClassifier()

In [58]:
predictions = rf.predict(test_features)

In [59]:
print("Accuracy: " + str(accuracy_score(test_labels, predictions)))
print("Recall for nothing: " + str(recall_score(test_labels, predictions, average="binary", pos_label="Nenhum")))
print("Recall for flooded: " + str(recall_score(test_labels, predictions, average="binary", pos_label="Alagamento")))
print(confusion_matrix(test_labels, predictions))

Accuracy: 0.9929328621908127
Recall for nothing: 1.0
Recall for flooded: 0.3333333333333333
[[  1   2]
 [  0 280]]


In [51]:
print("Len: " + str(len(predictions)))
alagamentos = []
nenhum = []
for f in predictions:
    if f == "Nenhum":
        nenhum.append(f)
    if f == "Alagamento":
        alagamentos.append(f)

print("Len alagamentos: " + str(len(alagamentos)))
print("Len nenhum: " + str(len(nenhum)))

Len: 113
Len alagamentos: 0
Len nenhum: 113


Verificar a porcentagem de flooded

In [ ]:

df_merged.groupby(['flooded']).count().plot(kind='pie', y="geometry", autopct='%1.0f%%')

In [ ]:
len(df_merged[df_merged["hex"] == "88a831acc5fffff"])